<a href="https://colab.research.google.com/github/PhoebeLu1011/1132code/blob/main/week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://ai.google.dev/

https://ai.google.dev/gemini-api/docs/quickstart?lang=python

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [4]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1rDQ19KEUqfFNpKMZq4Ga1SfH6r1xuJyGo5AIxumDDMw/edit?usp=sharing')

In [5]:
gsheets

<Spreadsheet '113-2codeWeek2' id:1rDQ19KEUqfFNpKMZq4Ga1SfH6r1xuJyGo5AIxumDDMw>

In [7]:
from gspread_dataframe import set_with_dataframe

# 選擇要更新的工作表（選擇第一個工作表）
worksheet = gsheets.get_worksheet(0)  # 如果是其他工作表，可以用 get_worksheet(index)

In [8]:
worksheet

<Worksheet '工作表1' id:0>

In [6]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [10]:
# 取得 Google Sheets 內容
data = worksheet.get_all_values()  # 取得所有儲存格資料，回傳為 2D 陣列
df = pd.DataFrame(data)

In [11]:
df

,0,1,2
0,Category,Activity,Calories
1,cardio,Running,400
2,cardio,Cycling,350
3,strength training,Weightlifting,200
4,strength training,Push-ups,100
5,sports,Basketball,300
6,sports,Soccer,320


In [31]:
prompt = f'請你把以下內容進行摘要 {df.to_string()} 並且提供給一位久坐的上班族每周運動量的建議與Activity，並幫忙排每周運動課表菜單，直接幫我生成，不用回應我'

In [32]:
prompt

'請你把以下內容進行摘要                    0              1         2\n0           Category       Activity  Calories\n1             cardio        Running       400\n2             cardio        Cycling       350\n3  strength training  Weightlifting       200\n4  strength training       Push-ups       100\n5             sports     Basketball       300\n6             sports         Soccer       320 並且提供給一位久坐的上班族每周運動量的建議與Activity，並幫忙排每周運動課表菜單，直接幫我生成，不用回應我'

In [33]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content(prompt)
print(response.text)

## 摘要

這份資料記錄了不同種類的運動 (Category) 及其活動 (Activity) 所消耗的卡路里 (Calories)。其中，有氧運動 (cardio) 包括跑步和自行車，力量訓練 (strength training) 包括舉重和俯臥撐，運動 (sports) 則有籃球和足球。

## 給久坐上班族的運動建議與課表 (每周)

**運動建議：**

*   **目標：** 提升心肺功能、增強肌肉力量、改善體態、舒緩壓力。
*   **頻率：** 每週至少進行 3 天的運動，並確保每天有足夠的休息時間。
*   **強度：** 從低強度開始，逐漸增加運動強度和時間。
*   **注意事項：** 運動前進行熱身，運動後進行伸展。注意補充水分，如有不適請立即停止。

**每週運動課表菜單：**

**星期一：**

*   **有氧運動 (30 分鐘):** 快走 ( brisk walking ) 或 慢跑 ( jogging )。目標： 讓心跳加速、微微出汗。
*   **伸展 (10 分鐘):** 拉伸腿部、背部、肩部等部位。

**星期二：**

*   **力量訓練 (20 分鐘):**
    *   **深蹲 ( Squats ):** 3 組，每組 10-12 次。
    *   **俯臥撐 ( Push-ups ):** 3 組，盡可能多做。如果太難，可以從跪姿俯臥撐開始。
    *   **弓箭步 ( Lunges ):** 每條腿 3 組，每組 10-12 次。
    *   **平板支撐 ( Plank ):** 3 組，每組保持 30 秒。
*   **伸展 (10 分鐘):** 拉伸手臂、胸部、腿部等部位。

**星期三：**

*   **休息：** 讓身體恢復。可以做一些輕鬆的活動，例如散步或瑜珈。

**星期四：**

*   **有氧運動 (30 分鐘):** 自行車 ( Cycling ) 或 游泳 ( Swimming )。
*   **伸展 (10 分鐘):** 拉伸腿部、背部、肩部等部位。

**星期五：**

*   **力量訓練 (20 分鐘):**
    *   **硬舉 ( Deadlift ):** 3 組，每組 8-10 次。(如果沒有啞鈴，可以利用裝滿水的寶特瓶)
    *   **划船

In [34]:
type(response.text)

str

In [35]:
from io import StringIO

# 轉換為 DataFrame
df_all = pd.read_csv(StringIO(response.text))

In [36]:
df_all

,## 摘要
0,這份資料記錄了不同種類的運動 (Category) 及其活動 (Activity) 所消耗的...
1,## 給久坐上班族的運動建議與課表 (每周)
2,**運動建議：**
3,* **目標：** 提升心肺功能、增強肌肉力量、改善體態、舒緩壓力。
4,* **頻率：** 每週至少進行 3 天的運動，並確保每天有足夠的休息時間。
5,* **強度：** 從低強度開始，逐漸增加運動強度和時間。
6,* **注意事項：** 運動前進行熱身，運動後進行伸展。注意補充水分，如有不適請立即停止。
7,**每週運動課表菜單：**
8,**星期一：**
9,* **有氧運動 (30 分鐘):** 快走 ( brisk walking ) 或 慢...


In [40]:
df_all

,## 摘要
1,## 給久坐上班族的運動建議與課表 (每周)
2,**運動建議：**
3,* **目標：** 提升心肺功能、增強肌肉力量、改善體態、舒緩壓力。
4,* **頻率：** 每週至少進行 3 天的運動，並確保每天有足夠的休息時間。
5,* **強度：** 從低強度開始，逐漸增加運動強度和時間。
6,* **注意事項：** 運動前進行熱身，運動後進行伸展。注意補充水分，如有不適請立即停止。
7,**每週運動課表菜單：**
8,**星期一：**
9,* **有氧運動 (30 分鐘):** 快走 ( brisk walking ) 或 慢...
10,* **伸展 (10 分鐘):** 拉伸腿部、背部、肩部等部位。


In [41]:
#選擇要更新的工作表（選擇第一個工作表）
worksheet_allcommand = gsheets.get_worksheet(3)  # 如果是其他工作表，可以用 get_worksheet(index)

worksheet_allcommand.clear()
# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet_allcommand, df_all)